In [ ]:
fdiry = "/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data"
print(os.listdir(fdiry))

lst = list()
CHROMS = np.r_[np.arange(1,23), ["X"]]
for chrom in CHROMS:
    ###
    fname = "chr" + chrom + ".fa"
    fpath = os.path.join(fdiry, fname)
    print(fpath, flush = True)
    dat = IOHelper.get_fastas_from_file(fpath, uppercase=True)
    
    ###
    lst.append(dat)

dat_fasta = pd.concat(lst).reset_index()
print(dat_fasta.head())


sequence_length = len(dat.sequence[0])
print(f"Sequence Length: {sequence_length}")

# Convert sequence to one hot encoding matrix
seq_matrix = SequenceHelper.do_one_hot_encoding(
    dat_fasta.sequence, 
    sequence_length,
    SequenceHelper.parse_alpha_to_seq)


X = np.nan_to_num(seq_matrix) # Replace NaN with zero and infinity with large finite numbers
X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))


fdiry = "/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results"
lst = list()

for chrom in CHROMS:
    ###
    fname = "dat_score_chr" + chrom + ".tsv"
    fpath = os.path.join(fdiry, fname)
    print(fpath)
    dat = pd.read_table(fpath)
    
    ###
    lst.append(dat)

dat_activity = pd.concat(lst)
print(dat_activity)

In [1]:
source ../config_duke.sh -v

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
PATH OF SOURCE:     /hpc/group/reddylab/Kuei/source
PATH OF EXECUTABLE: /hpc/group/reddylab/Kuei/exe
PATH OF ANNOTATION: /hpc/group/reddylab/Kuei/annotation
PATH OF PROJECT:    /hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /work/kk319/out/proj_combeffect



In [2]:
${FD_PRJ}/sing_proj_combeffect.sh ls

cradle_peak_00igv.ipynb
cradle_peak_01resize.ipynb
cradle_peak_02count.ipynb
cradle_peak_03table_sing.ipynb
cradle_peak_04seq.ipynb
cradle_peak_04seq_test.ipynb
cradle_peak_11deepstarr_train_01chr17.ipynb
cradle_peak_11deepstarr_train_02wg.ipynb
cradle_peak_12deepstarr_score_01chr17.ipynb
cradle_peak_12deepstarr_score_02wg.ipynb
cradle_peak_13deepstarr_motif_01chr17.ipynb
cradle_peak_13deepstarr_motif_02wg.ipynb
EDA_peaks.ipynb
model
Neural_Network_DNA_Demo
out.bed
test.bed
test.fa
test.fa.fai
test_run_train_script.ipynb


In [10]:
cat deepstarr_01train.py

### set environment
import sys
sys.path.append('/home/mount/project/')
from config_sing import *

### import basic 
import tensorflow as tf
import json
from sklearn.model_selection import train_test_split

#import keras
from tensorflow import keras
import tensorflow.keras.layers as kl
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import Dropout, Reshape, Dense, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization, InputLayer, Input
from tensorflow.keras import models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, History, ModelCheckpoint

### set GPU for tensorflow
physical_devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(physical_devices)

### from https://github.com/const-ae/Neural_Network_DNA_Demo
sys.path.append('Neural_Network_DNA

In [7]:
source ../config_duke.sh
# scanvager-gpu

sbatch -A chsi -p chsi-gpu --gres=gpu:1 --mem=10G \
    -o ${FD_LOG}/deepstarr_01train_out.txt \
    -e ${FD_LOG}/deepstarr_01train_err.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
${FD_PRJ}/sing_deepstarr.sh python deepstarr_01train.py \
    --params ./model/test.json \
    --prefix model_test \
    --fout   /home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_results

EOF

Submitted batch job 26813315


In [2]:
cat ${FD_LOG}/deepstarr_01train_out.txt

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Arguements
params: ./model/test.json
fout: /home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_results
prefix: test
Parameters
batch_size: 128
epochs: 5
early_stop: 10
kernel_size1: 7
kernel_size2: 3
kernel_size3: 5
kernel_size4: 3
lr: 0.002
num_filters: 256
num_filters2: 60
num_filters3: 60
num_filters4: 120
n_conv_layer: 4
n_add_layer: 2
dropout_prob: 0.4
dense_neurons1: 256
dense_neurons2: 256
pad: same
Load data
(87909, 1000, 4)
(87909, 2)
(70327, 1000, 4)
(17582, 1000, 4)
(70327, 2)
(17582, 2)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1000, 4)]    0                                            
__________________________________________________________________________________________________
Conv1D_1st 

In [36]:
cat ${FD_LOG}/deepstarr_01train_err.txt

2022-01-21 15:48:07.452830: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2022-01-21 15:48:07.461134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-21 15:48:07.461456: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla V100-PCIE-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:13:00.0
2022-01-21 15:48:07.675109: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2022-01-21 15:48:08.303444: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
2022-01-21 15:48:08.774756: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcu

In [18]:
source ../config_duke.sh
# scanvager-gpu

sbatch -A chsi -p chsi-gpu --gres=gpu:1 --mem=10G \
    -o ${FD_LOG}/deepstarr_02score_out.txt \
    -e ${FD_LOG}/deepstarr_02score_err.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
${FD_PRJ}/sing_deepstarr.sh python deepstarr_02score.py

EOF

Submitted batch job 26835040


In [22]:
cat ${FD_PRJ}/sing_proj_combeffect.sh ls

#!/bin/bash

#source config_duke.sh

# this script launches the singularity service for project combeffect

#FD_HOME=/gpfs/fs1/data/reddylab/Kuei/singularity/home
#FD_WORK=/gpfs/fs1/data/reddylab/Kuei
#FD_GITREPO=/gpfs/fs1/data/reddylab/Kuei/GitRepo
#FD_REDDYLAB=/gpfs/fs1/data/reddylab

### Stackoverflow: how-can-i-get-the-source-directory-of-a-bash-script-from-within-the-script-itself
SCRIPT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
source ${SCRIPT_DIR}/config_duke.sh

#singularity exec \
#    -H ${PWD}:/home \
#    -B ${FD_WORK}:/mount/work \
#    -B ${FD_PRJ}:/mount/project \
#    -B ${FD_RLAB}:/mount/reddylab \
#    ${FD_SING}/singularity_proj_combeffect.sif "$@"

singularity exec \
    -H ${PWD}:/home \
    -B ${FD_WORK}:/home/mount/work \
    -B ${FD_PRJ}:/home/mount/project \
    -B ${FD_RLAB}:/home/mount/reddylab \
    ${FD_SING}/singularity_proj_combeffect.sif "$@"
    
#rm -rf ${PWD}/mount
